<a href="https://colab.research.google.com/github/louzeiro/Roteamento-Veiculo/blob/main/Ortools_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook desenvolvido durante os estudos da ferramenta Or-Tools<br>
Seção: Roteamento<br>
Subseção: Traveling Routing Problem


Fonte: https://developers.google.com/optimization/routing/tsp


# Definição do problema

Nesta seção foi apresentada a implementação para a solução do problema do caixeiro viajante com o Or-Tools. No exemplo de uso, foi definido o conjunto de pontos do problema, conforme a imagem abaixo.
<p>
  <a href="exemplo">
    <img src="https://developers.google.com/optimization/images/routing/tsp_map.png">
  </a>
</p>


# Conjunto de dados

In [ ]:
"""
 Dados do problema: conjunto das distâncias entre os pontos, incluindo o
 deposito. Nesse caso, não há diferênça no percurso de ida e volta.
 O ponto de partida e de chegada será o depósito.
"""
def create_data_model():
    data = {} 
    data['distance_matrix'] = [ # matriz de distância entre os pontos
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972], # ponto 1
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],   # ponto 2
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],     # ponto 3
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],   # ponto 4
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],       # ponto 5
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],     # ponto 6
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701], # ponto 7
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],# ponto 8
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],   # ponto 9
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],    # ponto 10
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],  # ponto 11
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],   # ponto 12
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],     # ponto 13
    ]  
    data['num_vehicles'] = 1 # total de veículos usados na solução
    data['depot'] = 0        # identificação da origem, nesse caso o depósito
    return data

# Definindo o modelo de roteamento

In [ ]:
#import os
#os.system('pip install ortools') #instalando a biblioteca no ambiente de desenvolvimento

from ortools.constraint_solver import pywrapcp 
data = create_data_model()                                           # carregando os dados do problema
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), # número de pontos
                                       data['num_vehicles'],         # total de veículos/caixeiros  
                                       data['depot'])                # índece da origem/depósito
routing = pywrapcp.RoutingModel(manager)                             # criando o modelo


# Defindo a função que retornará a distância entre dois pontos

Essa função é necessária, pois o Ortools trabalha com um sistema diferente de referência.

In [ ]:
"""Retorna a distância entre os pontos"""
def distance_callback(from_index, to_index):
  from_node = manager.IndexToNode(from_index)   # convertendo o index interno do ortools
  to_node = manager.IndexToNode(to_index)       # para o index de acesso à matrix
  return data['distance_matrix'][from_node][to_node]

"""Realiza a chamada e a registra no solve """
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [ ]:
data['distance_matrix'][1][2]

1745

# Definindo o custo

Neste exemplo será utilizado a distância entre os pontos como o custo do problema. Assim, será utilizada a variável *transit_callback_index* que é a referência interna do solve para a função *distance_callback*, que retorna a distância.

In [ ]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Definindo os parâmetros e a Heurística da solução


In [ ]:
from ortools.constraint_solver import routing_enums_pb2

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Exibindo os resultados obtidos

In [ ]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Custo da solução: {} metros'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Rota para o veículo 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    plan_output += 'Rota para a distância percorrida: {} metros\n'.format(route_distance)

    print(plan_output)

# Main

In [ ]:
solution = routing.SolveWithParameters(search_parameters)

if solution:
    print_solution(manager, routing, solution)

Custo da solução: 7569 metros
Rota para o veículo 0:
 0 -> 7 -> 2 -> 3 -> 6 -> 8 -> 1 -> 11 -> 12 -> 4 -> 5 -> 10 -> 9 -> 0
Rota para a distância percorrida: 7569 metros



# Salvando as rotas em um array

In [ ]:
def get_routes(solution, routing, manager):
  routes = [] 
  for route_nbr in range(routing.vehicles()): # para cada veículo do conjunto de rotas
    index = routing.Start(route_nbr)      # Ponto de partida (depósito)
    route = [manager.IndexToNode(index)]  # Convertende e adicionando o index no array
    while not routing.IsEnd(index):       # Enquanto não for o último índice
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index)) 
    routes.append(route)
  return routes

#exibindo a rota final
routes = get_routes(solution, routing, manager)
for i, route in enumerate(routes):
  print('Rota obtida com a solução para o veículo', i,'=', route)

Rota obtida com a solução para o veículo 0 = [0, 7, 2, 3, 6, 8, 1, 11, 12, 4, 5, 10, 9, 0]
